In [39]:
import tensorflow as tf
import sys
print(sys.version)
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

3.5.2 |Continuum Analytics, Inc.| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]


In [93]:
class MF_RS():
    def __init__(self, numUsers, numSongs, embedding_dim):
        self.batch_size = 5;
        self.numUsers = numUsers
        self.numSongs = numSongs
        self.epochs = 100
        self.userMat = tf.Variable(tf.random_normal([numUsers, embedding_dim]))
        self.songMat = tf.Variable(tf.random_normal([numSongs, embedding_dim]))
        self.users = tf.placeholder(tf.int32, shape =(self.batch_size))
        self.songs = tf.placeholder(tf.int32, shape =(self.batch_size))
        self.rating = tf.placeholder(tf.float32, shape = (self.batch_size))
        self.U = tf.nn.embedding_lookup(self.userMat, self.users)
        self.W = tf.nn.embedding_lookup(self.songMat, self.songs)
        self.yhat = tf.reduce_sum(tf.mul(self.U, self.W), 1)
        self.cost = tf.reduce_mean(tf.nn.l2_loss(self.yhat - self.rating))
        self.optimizer = tf.train.GradientDescentOptimizer(learning_rate = .01).minimize(self.cost)
        self.session = tf.Session()
        self.session.run(tf.initialize_all_variables())
    def train(self, users, songs, ratings):
        for i in range(self.epochs):
            avg_cost = 0
            perm = np.random.permutation(len(ratings))
            num_batches = len(ratings) // self.batch_size
            for b_idx in range(num_batches):
                batch = perm[self.batch_size * b_idx:self.batch_size * (b_idx + 1)]
                if b_idx == num_batches - 1:
                    batch = perm[self.batch_size * b_idx:]
                users_batch = users[batch]
                songs_batch = songs[batch]
                ratings_batch = ratings[batch]
                avg_cost += model.session.run([model.cost, model.optimizer],
                                  {model.users:users_batch, model.songs:songs_batch, model.rating:ratings_batch})[0]
            print(avg_cost/num_batches)

In [98]:
a = np.array([1, 2, 3, 4, 5])
b = np.array([1, 2, 3, 4, 5])
c = np.array([4, 3, 2, 5, 1])
model = MF_RS(10, 10, 7)
for i in range(10):
    print(model.session.run([model.cost, model.optimizer], {model.users:a, model.songs:b, model.rating:c}))


[30.73735, None]
[22.543304, None]
[16.707861, None]
[12.49137, None]
[9.4120789, None]
[7.1439009, None]
[5.4604683, None]
[4.2020283, None]
[3.2546062, None]
[2.5362709, None]


In [99]:
model.train(a, b, c)

1.98777019978
1.56603336334
1.23958039284
0.98527097702
0.785988271236
0.62898170948
0.504684507847
0.405862897635
0.327004462481
0.263877272606
0.213207408786
0.172445237637
0.139592349529
0.113073408604
0.0916408970952
0.0743014663458
0.0602623671293
0.0488880053163
0.0396680608392
0.0321914292872
0.026126794517
0.0212062671781
0.0172133333981
0.0139727406204
0.0113424547017
0.0092073911801
0.00747432699427
0.0060674380511
0.0049253818579
0.00399825721979
0.00324563891627
0.00263468595222
0.00213872990571
0.00173612660728
0.00140930735506
0.00114401197061
0.00092864461476
0.000753832457121
0.000611923867837
0.000496737251524
0.000403242127504
0.000327337067574
0.000265723472694
0.000215706328163
0.000175105466042
0.000142151257023
0.000115396192996
9.3680806458e-05
7.60533221182e-05
6.17399346083e-05
5.01237082062e-05
4.0694103518e-05
3.30369257426e-05
2.68209278147e-05
2.17762317334e-05
1.76803296199e-05
1.43543584272e-05
1.16540659292e-05
9.46209183894e-06
7.68285281083e-06
6.23731

In [100]:
movieratings = pd.read_csv('ratings.csv')

In [101]:
print(len(movieratings['userId'].unique()))
print(len(movieratings['movieId'].unique()))

671
9066


In [102]:
model = MF_RS(10, 10, 7)

In [103]:
movieratings.describe()

,userId,movieId,rating,timestamp
count,100004.000000,100004.000000,100004.000000,1.000040e+05
mean,347.011310,12548.664363,3.543608,1.129639e+09
std,195.163838,26369.198969,1.058064,1.916858e+08
min,1.000000,1.000000,0.500000,7.896520e+08
25%,182.000000,1028.000000,3.000000,9.658478e+08
50%,367.000000,2406.500000,4.000000,1.110422e+09
75%,520.000000,5418.000000,4.000000,1.296192e+09
max,671.000000,163949.000000,5.000000,1.476641e+09


In [104]:
def getDfSummary(input_data):
    output_data = input_data.describe(include = 'all').T
    var = pd.DataFrame(data = {'nanvals': pd.Series(), 'number_distinct': pd.Series()})
    for i in range(len(input_data.columns)):
        nanvals = input_data.ix[:,i].isnull().sum()
        number_distinct = len(input_data.ix[:,i].value_counts())
        var = var.append(pd.DataFrame([[nanvals, number_distinct]], columns = ['nanvals', 'number_distinct']))
    var.index = output_data.index.values
    output_data['nanvals'] = var['nanvals']
    output_data['number_distinct'] = var['number_distinct']
    return output_data
output_data = getDfSummary(movieratings)

In [105]:
output_data

,count,mean,std,min,25%,50%,75%,max,nanvals,number_distinct
userId,100004.0,3.470113e+02,1.951638e+02,1.0,182.0,3.670000e+02,5.200000e+02,6.710000e+02,0.0,671.0
movieId,100004.0,1.254866e+04,2.636920e+04,1.0,1028.0,2.406500e+03,5.418000e+03,1.639490e+05,0.0,9066.0
rating,100004.0,3.543608e+00,1.058064e+00,0.5,3.0,4.000000e+00,4.000000e+00,5.000000e+00,0.0,10.0
timestamp,100004.0,1.129639e+09,1.916858e+08,789652009.0,965847824.0,1.110422e+09,1.296192e+09,1.476641e+09,0.0,78141.0


In [117]:
users = movieratings.ix[:,0].values
songs = movieratings.ix[:,1].values
ratings = movieratings.ix[:,2].values

In [118]:
songmodel = MF_RS(671, 9066, 100)

In [119]:
print(len(users), len(songs), len(ratings))

100004 100004 100004


In [120]:
songs

array([  31, 1029, 1061, ..., 6365, 6385, 6565])

In [121]:
#this is failing
songmodel.train(users, songs, ratings)

InvalidArgumentError: indices[0] = 1207 is not in [0, 10)
	 [[Node: embedding_lookup_75 = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, _class=["loc:@Variable_75"], validate_indices=true, _device="/job:localhost/replica:0/task:0/cpu:0"](Variable_75/read, _recv_Placeholder_112_0)]]
Caused by op 'embedding_lookup_75', defined at:
  File "/home/lee/anaconda3/envs/tf/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/lee/anaconda3/envs/tf/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/lee/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/lee/anaconda3/envs/tf/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/lee/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/home/l